In [1]:
"""
Sample from a trained model
"""
import os
import pickle
from contextlib import nullcontext
import torch
import tiktoken
from model import GPTConfig, GPT

# -----------------------------------------------------------------------------
init_from = 'resume' # either 'resume' (from an out_dir) or a gpt2 variant (e.g. 'gpt2-xl')
out_dir = 'out-stock' # ignored if init_from is not 'resume'
start = "\n" # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"
num_samples = 10 # number of samples to draw
max_new_tokens = 500 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
device = 'cpu' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster
# -----------------------------------------------------------------------------


In [2]:

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cpu' # for later use in torch.autocast
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# model
if init_from == 'resume':
    # init from a model saved in a specific directory
    ckpt_path = os.path.join(out_dir, 'ckpt.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    gptconf = GPTConfig(**checkpoint['model_args'])
    model = GPT(gptconf)
    state_dict = checkpoint['model']
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)


model.eval()
model.to(device)


# look for the meta pickle in case it is available in the dataset folder
load_meta = False
if init_from == 'resume' and 'config' in checkpoint and 'dataset' in checkpoint['config']: # older checkpoints might not have these...
    meta_path = os.path.join('data', checkpoint['config']['dataset'], 'meta.pkl')
    load_meta = os.path.exists(meta_path)
if load_meta:
    print(f"Loading meta from {meta_path}...")
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    # TODO want to make this more general to arbitrary encoder/decoder schemes
    stoi, itos = meta['stoi'], meta['itos']
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])


number of parameters: 304.11M
Loading meta from data/stock/meta.pkl...


In [3]:
import numpy as np
import pandas as pd

np.random.seed(42)

# 生成50x32大小的数组，值范围在1到10之间
data = np.random.rand(33, 2048) + 1

# 将NumPy数组转换为pandas DataFrame
data = pd.DataFrame(data)
data = torch.tensor(data.values, dtype=torch.float16)
data = data.unsqueeze(0)
print(f'data is {data}')

eval_X = data[:, :32, :]
eval_Y = data[:, -1, :]


print(f'X is {eval_X}, X.shape is {eval_X.shape}')
print(f'Y is {eval_Y}, Y.shape is {eval_Y.shape}')
value, index = torch.topk(eval_Y, 10, -1)
print(f'value is {value}')
print(f'index is {index}')



data is tensor([[[1.3750, 1.9512, 1.7324,  ..., 1.7227, 1.0674, 1.7080],
         [1.5439, 1.0820, 1.4580,  ..., 1.9287, 1.3320, 1.4658],
         [1.0137, 1.0820, 1.2588,  ..., 1.3613, 1.1084, 1.8760],
         ...,
         [1.2646, 1.0977, 1.5430,  ..., 1.5859, 1.7656, 1.8799],
         [1.8291, 1.1279, 1.7441,  ..., 1.4326, 1.8789, 1.4580],
         [1.3428, 1.6299, 1.6191,  ..., 1.8438, 1.6211, 1.6465]]],
       dtype=torch.float16)
X is tensor([[[1.3750, 1.9512, 1.7324,  ..., 1.7227, 1.0674, 1.7080],
         [1.5439, 1.0820, 1.4580,  ..., 1.9287, 1.3320, 1.4658],
         [1.0137, 1.0820, 1.2588,  ..., 1.3613, 1.1084, 1.8760],
         ...,
         [1.8516, 1.4912, 1.2881,  ..., 1.7627, 1.6074, 1.6445],
         [1.2646, 1.0977, 1.5430,  ..., 1.5859, 1.7656, 1.8799],
         [1.8291, 1.1279, 1.7441,  ..., 1.4326, 1.8789, 1.4580]]],
       dtype=torch.float16), X.shape is torch.Size([1, 32, 2048])
Y is tensor([[1.3428, 1.6299, 1.6191,  ..., 1.8438, 1.6211, 1.6465]],
       dtyp

In [4]:

# run generation
with torch.no_grad():
    with ctx:  
        probs, index = model.generate_last(eval_X, top_k=top_k)
        print(f'predict index is {index}, index.shape is {index.shape}')
        print(f'predict probs is {probs}, probs.shape is {probs.shape}')
        print('---------------')


predict index is tensor([[ 111,  378,  491,   70,  257,   63,  174,  722,  625,  931,  477,  567,
          592,  193,  878,  199,  304,  915,  744,  770,  889,  589,  468,  515,
          599,  699,  119,  531,  427,  446,   32,  517,  757,  781,  855,  488,
          841, 1014,  998,  605,  702,  634,  650,  284,  615,  681, 1002,   62,
          946,  740,  518,  212,  810,  647,  907,  358,  670,  289,  887,  402,
          172,  248,  945,  315,  825,  311,   96,  626,  750,  167,  754,  158,
          811,  299,  717,    2,   18,  559,  904,  109,  463,  732,  388,  457,
          916,  927,  105,  718,  355,  272,    7,  903,  636,  787,  867,  322,
           50,  780,   37,  643,  761,  723,  259,   35,  510, 1008,  241,  769,
          440,  783,  376,  928,  649,  177,  392,  145,  874,  412,  914,  772,
          250,  264,  863,  337,   66,  560,  252,  314,  657,  848,  991,  467,
           73,  957,  977,   30,  335,  204,  972,  503,  826,  812,  281,  347,
           